In [ ]:
# Install libraries or dependencies
!pip install -q condacolab
import condacolab
condacolab.install()
!pip install biopython==1.79
!conda install -c conda-forge pymol-open-source scipy matplotlib psycopg2-binary aria2p python-dotenv 
#TODO: add biopython version 1.78 (because may be upgraded . 1.8 and up not good).
#TODO: add for all - versions!!!
# !pip install -r requirements.txt

In [ ]:
#@title Environment Setup and Imports

# Install Essential System Packages and GPG Tools
!sudo apt install dirmngr ca-certificates software-properties-common gnupg gnupg2 apt-transport-https curl -y

# Install PostgresSQL
!curl -fSsL https://www.postgresql.org/media/keys/ACCC4CF8.asc | gpg --dearmor | sudo tee /usr/share/keyrings/postgresql.gpg > /dev/null
!echo 'deb [arch=amd64,arm64,ppc64el signed-by=/usr/share/keyrings/postgresql.gpg] http://apt.postgresql.org/pub/repos/apt/ jammy-pgdg main' | sudo tee /etc/apt/sources.list.d/pgdg.list
!sudo apt update
!sudo apt install postgresql-client-15 postgresql-15 -y
!sudo service postgresql start

# Create a new PostgreSQL database if it does not already exist
!sudo -u postgres psql -tc "SELECT 1 FROM pg_database WHERE datname = 'zincsight_pipeline_db';" | grep -q 1 || sudo -u postgres psql -c "CREATE DATABASE zincsight_pipeline_db;"
!sudo -u postgres psql -c "GRANT ALL PRIVILEGES ON DATABASE zincsight_pipeline_db TO postgres;"
!sudo -u postgres psql -c "ALTER USER postgres WITH PASSWORD 'postgres';"

!sudo apt install pigz

In [ ]:
# Install aria2 for optional batch downloads
!sudo apt install aria2 -y

In [ ]:
# Clone the GitHub repository and change into the repo directory
!git clone https://github.com/MECHTI1/ZincSight.git
%cd ZincSight/

In [ ]:
# Add essential SQL tables to the established database
!sudo -u postgres psql -d zincsight_pipeline_db -P pager=off -f "src/setup_pg_db_with_tables/PostgreSQL_4_necessary_tables.sql"
#TODO: maybe take out sudo

In [ ]:
# mounted_to_drive=True
# if mounted_to_drive:
#   # Mount Google Drive
#   from google.colab import drive
#   drive.mount('/content/drive/')
#TODO: MAYBE OMMIT

In [ ]:
import ipywidgets as widgets
import os

In [ ]:
# Define the input structure IDs for the textarea input widget
input_test_31_af_ids = 'A0A068N621, A0A0F6AZI6, A0A292DHH8, A0A2U3D0N8, A0A3F2YM30, A0A5H1ZR49, G8ZFK7, O60232, P0A6G5, P0DUH5, P37659, P38164, Q03760, Q08281, Q2K0Z2, Q2UFA9, Q5W0Q7, Q66K64, Q68EN5,Q6CXX6, Q7MVV4, Q86T03, Q8N8R7, Q8NBJ9, Q96JC1, Q9BWG6,Q9D1N4, Q9KP27, Q9M1V3, Q9NUN7, Q9NXF7'

# Define Textarea for inputting structure IDs
structure_ids_input = widgets.Textarea(
    value=f"{input_test_31_af_ids}",
    description='',
    layout=widgets.Layout(width='100%', height='100px')
)

# Define a button to save Textarea content to a file
save_button = widgets.Button(
    description="Save Structure IDs",
    button_style="success"
)

# Define a FileUpload widget for file uploads
file_upload = widgets.FileUpload(
    accept='.cif,.pdb,.tar,.tar.gz,.txt',  # Restrict accepted file types
    multiple=True,  # Allow multiple file uploads
    description='Upload files'
)

     

In [ ]:

# Function to save structure IDs from Textarea to a file
def save_text_structures_ids(_):
    target_dir = 'text_structures_ids_to_download'
    os.makedirs(target_dir, exist_ok=True)

    # Define the file name and file path
    file_name = 'structure_ids.txt'
    file_path = os.path.join(target_dir, file_name)

    # Write the content of the Textarea widget to the file
    with open(file_path, 'w') as f:
        f.write(str(structure_ids_input.value))

    print(f"Structure IDs successfully saved to {file_path}")

# Function to save uploaded files
def save_uploaded_files(change):
    target_dir = 'Query AlphaFold structures'
    os.makedirs(target_dir, exist_ok=True)

    # Save each uploaded file to the target directory
    for file_name, file_info in change['new'].items():
        file_path = os.path.join(target_dir, file_name)
        with open(file_path, 'wb') as f:
            f.write(file_info['content'])
    print(f"Files successfully uploaded to {target_dir}")

     

In [ ]:

# Attach the save function to the button click
save_button.on_click(save_text_structures_ids)

# Automatically save uploaded files
file_upload.observe(save_uploaded_files, names='value')

# Organize widgets in an Accordion
accordion = widgets.Accordion(
    children=[widgets.VBox([structure_ids_input, save_button]), file_upload],
    selected_index=None  # Keep all sections collapsed by default
)

# Set titles for the accordion sections
accordion.set_title(0, 'Write structure IDs (separated by commas)')
accordion.set_title(1, 'Upload structure files (.cif, .pdb, .tar, .tar.gz, .txt)')

# Display the accordion
accordion

In [ ]:
from main import execute_zincsight  # TODO: uncomment this row after gather the whole together and ready!!!!!!!

In [ ]:
# Define HTML with custom styling for larger text and no extra row
description_text = widgets.HTML(
    value="""
    <p style="font-size:16px; font-weight:bold; margin-bottom: 0;">
        Histidine Imidazole Rotamer Examination:
    </p>
    <p style="font-size:14px; margin-top: 0;">
        Choose 'Yes' to perform examination, which may improve sensitivity to misoriented histidines.
    </p>
    """
)

# Define the ToggleButtons widget with a concise description
toggle_buttons_wether_rotamer = widgets.ToggleButtons(
    options=['No', 'Yes'],
    description='',
    disabled=False,
    button_style='',
    tooltips=[
        'Yes: Include rotamer examination option',
        'No: No rotamer examination might reduce sensitivity to misoriented histidines'
    ]
)


# Initialize the global variable to track the rotamer examination option
selected_rot_option = False

def on_toggle_change(change):
    """Updates the global selected_rot_option based on the toggle's value."""
    global selected_rot_option
    selected_rot_option = True if change.new == "Yes" else False

def execute_main_algo(b):
    """Executes the main algorithm with selected rotamer option boolean argument."""
    global selected_rot_option

    print("Running ZincSight")

    execute_zincsight(selected_rot_option)  # TODO: Verify that line!!!!!!!

    #TODO: add seprately as a test, to verify NO/YES for rotamer is working -
    print (f"starting function with selected rotamer option boolean = {selected_rot_option}")


# Create the continue button
continue_button = widgets.Button(description="Continue", button_style='primary')

# Attach the event handlers
toggle_buttons_wether_rotamer.observe(on_toggle_change, names='value')
continue_button.on_click(execute_main_algo)

# Display the description and toggle buttons together in a vertical layout
widgets.VBox([description_text, toggle_buttons_wether_rotamer, continue_button])

In [ ]:
from google.colab import files
from src.settings import STRUCTURES_WITH_PREDICTED_ZN
files.download(STRUCTURES_WITH_PREDICTED_ZN)